In [7]:
import dspy
from dspy.predict import Retry
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
import importlib

importlib.reload(dspy)

# Configure the AzureOpenAI language model
azure_turbo = dspy.AzureOpenAI(api_base='https://bionlp-gpt4-wang.openai.azure.com/', api_key="a494edc84d714b6c8a12e7212974b793", api_version='2024-03-01-preview', model='gpt-35-turbo')
dspy.settings.configure(lm=azure_turbo)

In [8]:
import re

def eval_metric(true, prediction, trace=None):
    try:
        pred = prediction.answer
        matches = re.findall(r"\([A-D]\)", pred)
        parsed_answer = matches[-1] if matches else ""
        return parsed_answer == true.answer
    except:
        return False 

In [9]:
class MedMCQAInstance(dspy.Signature):
    
    question = dspy.InputField(desc="Question")
    options = dspy.InputField(desc="Answer choice options")
    answer = dspy.OutputField(desc="You should respond with one of (A), (B), (C), or (D)")

In [4]:
class MedMCQA_Answer(dspy.Module):
    """You are a helpful medical expert, and your task is to answer a multi-choice medical question. Please first think step-by-step and then choose the answer from the provided options."""
    
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(MedMCQAInstance)
    
    def forward(self, question, options):
        response = self.generate_answer(question=question, options=options)
      
        valid_response = "(A)" in response.answer or "(B)" in response.answer or "(C)" in response.answer or "(D)" in response.answer  

        dspy.Suggest(valid_response, "You must respond with one of (A), (B), (C), or (D) as part of your answer.")

        return dspy.Prediction(answer=response.answer)

In [5]:
import json

train_data = []
val_data = []
test_data = []

with open("/Users/khandekarns/Documents/DSPy/eligible_notes/medmcqa_train_eligible.json") as file:
    train = json.load(file)

    
        
with open("/Users/khandekarns/Documents/DSPy/eligible_notes/medmcqa_test_eligible.json") as file:
    test = json.load(file)



In [6]:
import random

def question_with_options(options):

   return "\n(A) " + options['opa'] + "\n(B) " + options['opb'] +  "\n(C) " + options['opc'] + "\n(D) " + options['opd']


def generate_dspy_examples(dataset):
   
   num_to_letter = {1: "A", 2: "B", 3: "C", 4: "D"}

   examples = []
   
   for i in range(len(dataset)):

      answer_idx = dataset[i]["cop"]

      answer_choice =  num_to_letter[answer_idx]
      
      example = dspy.Example({"question": dataset[i]['question'], "options": question_with_options(dataset[i]), "answer": "(" + answer_choice + ")"}).with_inputs("question", "options") 

      examples.append(example)

   return examples 


random.shuffle(train_data)

train_data = random.sample(train_data, 11000)

train_dspy_examples = generate_dspy_examples(train_data[:10000])
val_dspy_examples = generate_dspy_examples(train_data[10000:])
test_dspy_examples = generate_dspy_examples(val_data)

ValueError: Sample larger than population or is negative

In [10]:
from dspy.evaluate import Evaluate

evaluate_test = Evaluate(devset=test_dspy_examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=True)

NameError: name 'test_dspy_examples' is not defined

In [8]:
answerqa_with_assertions = assert_transform_module(MedMCQA_Answer().map_named_predictors(Retry), backtrack_handler) 
evaluate_test(answerqa_with_assertions)

Average Metric: 41 / 70  (58.6):   2%|▏         | 69/4183 [00:00<00:17, 235.06it/s]INFO:backoff:Backing off request(...) for 0.7s (openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': True, 'severity': 'medium'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}})


Backing off 0.7 seconds after 1 tries calling function <function AzureOpenAI.request at 0x114d52a20> with kwargs {}


INFO:backoff:Backing off request(...) for 1.7s (openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': True, 'severity': 'medium'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}})


Backing off 1.7 seconds after 2 tries calling function <function AzureOpenAI.request at 0x114d52a20> with kwargs {}


INFO:backoff:Backing off request(...) for 2.7s (openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': True, 'severity': 'medium'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}})


...

Backing off 3.9 seconds after 3 tries calling function <function AzureOpenAI.request at 0x114d52a20> with kwargs {}


INFO:backoff:Backing off request(...) for 2.2s (openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}})


Backing off 2.2 seconds after 4 tries calling function <function AzureOpenAI.request at 0x114d52a20> with kwargs {}


ERROR:backoff:Giving up request(...) after 5 tries (openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}})


BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}

In [9]:
azure_turbo.inspect_history()




You are a helpful medical expert, and your task is to answer a multi-choice medical question. Please first think step-by-step and then choose the answer from the provided options.

---

Follow the following format.

Question: Question

Options: Answer choice options

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: You should respond with one of (A), (B), (C), or (D)

---

Question: Most common type of Non-Hodgkin's lymphoma in the orbit:

Options: (A) B-cell (B) T-cell (C) NK-cell (D) Plasma cell

Reasoning: Let's think step by step in order to produce the answer. We know that Non-Hodgkin's lymphoma is a type of cancer that affects the lymphatic system. The orbit refers to the eye socket. In the case of Non-Hodgkin's lymphoma in the orbit, the most common type is usually derived from the B-cells, which are a type of white blood cell responsible for producing antibodies. Therefore, the answer is (A) B-cell.

Answer: (A) B-cell





"\n\n\nYou are a helpful medical expert, and your task is to answer a multi-choice medical question. Please first think step-by-step and then choose the answer from the provided options.\n\n---\n\nFollow the following format.\n\nQuestion: Question\n\nOptions: Answer choice options\n\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\n\nAnswer: You should respond with one of (A), (B), (C), or (D)\n\n---\n\nQuestion: Most common type of Non-Hodgkin's lymphoma in the orbit:\n\nOptions: (A) B-cell (B) T-cell (C) NK-cell (D) Plasma cell\n\nReasoning: Let's think step by step in order to\x1b produce the answer. We know that Non-Hodgkin's lymphoma is a type of cancer that affects the lymphatic system. The orbit refers to the eye socket. In the case of Non-Hodgkin's lymphoma in the orbit, the most common type is usually derived from the B-cells, which are a type of white blood cell responsible for producing antibodies. Therefore, the answer is (A) B-cell.\n\nAnswer:

In [ ]:
from tqdm import tqdm
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

tqdm._instances.clear()

config = dict(max_bootstrapped_demos=5, max_labeled_demos=0, num_candidate_programs=20, num_threads=6)
teleprompter = BootstrapFewShotWithRandomSearch(metric = eval_metric, **config)
answer_question_teacher = teleprompter.compile(student=assert_transform_module(MedMCQA_Answer().map_named_predictors(Retry), backtrack_handler), teacher = assert_transform_module(MedMCQA_Answer().map_named_predictors(Retry), backtrack_handler), trainset=train_dspy_examples, valset=val_dspy_examples)

In [ ]:
evaluate_test(answer_question_teacher)

In [ ]:
from dspy.teleprompt import MIPROv2

teleprompter = MIPROv2(prompt_model= azure_turbo, task_model=azure_turbo, metric=eval_metric, num_candidates=20, metric_threshold=None)
kwargs = dict(num_threads=3, display_progress=True, display_table=True)
compiled_prompt_opt = teleprompter.compile(MedMCQA_Answer(), trainset=train_dspy_examples, valset= val_dspy_examples, max_bootstrapped_demos=5, eval_kwargs=kwargs, requires_permission_to_run=False)